# Day 14: Extended Polymerization

## Part 1

The incredible pressures at this depth are starting to put a strain on your submarine. The submarine has polymerization equipment that would produce suitable materials to reinforce the submarine, and the nearby volcanically-active caves should even have the necessary input elements in sufficient quantities.

The submarine manual contains instructions for finding the optimal polymer formula; specifically, it offers a polymer template and a list of pair insertion rules (your puzzle input). You just need to work out what polymer would result after repeating the pair insertion process a few times.

For example:
```
NNCB

CH -> B
HH -> N
CB -> H
NH -> C
HB -> C
HC -> B
HN -> C
NN -> C
BH -> H
NC -> B
NB -> B
BN -> B
BB -> N
BC -> B
CC -> N
CN -> C
```

The first line is the polymer template - this is the starting point of the process.

The following section defines the pair insertion rules. A rule like ```AB -> C``` means that when elements ```A``` and ```B``` are immediately adjacent, element ```C``` should be inserted between them. These insertions all happen simultaneously.

So, starting with the polymer template ```NNCB```, the first step simultaneously considers all three pairs:

The first pair (```NN```) matches the rule ```NN -> C```, so element ```C``` is inserted between the first ```N``` and the second ```N```.
The second pair (```NC```) matches the rule ```NC -> B```, so element ```B``` is inserted between the ```N``` and the ```C```.
The third pair (```CB```) matches the rule ```CB -> H```, so element ```H``` is inserted between the ```C``` and the ```B```.
Note that these pairs overlap: the second element of one pair is the first element of the next pair. Also, because all pairs are considered simultaneously, inserted elements are not considered to be part of a pair until the next step.

After the first step of this process, the polymer becomes ```NCNBCHB```.

Here are the results of a few steps using the above rules:
```
Template:     NNCB
After step 1: NCNBCHB
After step 2: NBCCNBBBCBHCB
After step 3: NBBBCNCCNBBNBNBBCHBHHBCHB
After step 4: NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB
```

This polymer grows quickly. After step 5, it has length 97; After step 10, it has length 3073. After step 10, ```B``` occurs 1749 times, ```C``` occurs 298 times, ```H``` occurs 161 times, and ```N``` occurs 865 times; taking the quantity of the most common element (```B```, 1749) and subtracting the quantity of the least common element (```H```, 161) produces ```1749 - 161 = 1588```.

Apply 10 steps of pair insertion to the polymer template and find the most and least common elements in the result. What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?

In [1]:
from collections import Counter

In [2]:
def load_insertions(path):
    file = open(path,'r')
    lines = file.readlines()
    file.close()
    
    split_ind = lines.index("\n")
    lines = ([l.strip("\n") for l in lines]) 
    
    initial = lines[:split_ind]
    
    ins = lines[split_ind+1:]
    pairs_arr = []
    inserts_arr = []
    for i in ins:
        pairs, inserts = i.split(' -> ')
        pairs_arr.append(pairs)
        inserts_arr.append(inserts)
    
    return initial[0], pairs_arr, inserts_arr

In [3]:
def poly_ins(polymer, pairs, insertions, steps):
    
    print("Polymer has an initial length of "+str(len(polymer))+": "+polymer)
    print()
    
    freqs = Counter() # counter to count the individual pairs

    for a, b in zip(polymer, polymer[1:]): # extract pairs of elements and count the frequencies
        s = a + b
        freqs[s]+=1
    
        inserts = {} # empty dictionary for which new pairs will be included with each insertion

    for i in range(len(pairs)): # step through each of the pair insertion rules
        # populate the inserts dictionary with the resultant pairs from each insertion
        inserts[pairs[i]] = [pairs[i][0]+insertions[i], insertions[i]+pairs[i][1]]
        
    counts = Counter(polymer) # count the individual elements in the polymer

    for _ in range(steps):
        tmp_ctr = Counter() # set up counter for each insertion step
        for pair, count in freqs.items(): # get each pair and counts 
            if pair in inserts:    # check for required insertions
                counts[inserts[pair][0][1]] += count # iterate the counter for each element in the pair
                for new_pair in inserts[pair]:
                    tmp_ctr[new_pair] += count # iterate the counter for each element in the new pairs
        freqs = tmp_ctr 
        print("Step "+str(_+1)+": Polymer has a length of "+str(sum(counts.values()))+"")
        
        
    mostCommon = counts.most_common()
    print()
    print("Most common element is "+mostCommon[0][0]+" with "+str(mostCommon[0][1])+" entries.")  
    print("The least common element is "+mostCommon[-1][0]+" with "+str(mostCommon[-1][1])+" entries.")
    print("The solution is: "+str(mostCommon[0][1])+" - "+str(mostCommon[-1][1])+" = "+str(mostCommon[0][1]-mostCommon[-1][1])+".")

In [4]:
p_init, pairs, inserts = load_insertions("day14_data_test.txt")

poly_ins(p_init, pairs, inserts, 10)

Polymer has an initial length of 4: NNCB

Step 1: Polymer has a length of 7
Step 2: Polymer has a length of 13
Step 3: Polymer has a length of 25
Step 4: Polymer has a length of 49
Step 5: Polymer has a length of 97
Step 6: Polymer has a length of 193
Step 7: Polymer has a length of 385
Step 8: Polymer has a length of 769
Step 9: Polymer has a length of 1537
Step 10: Polymer has a length of 3073

Most common element is B with 1749 entries.
The least common element is H with 161 entries.
The solution is: 1749 - 161 = 1588.


In [5]:
p_init, pairs, inserts = load_insertions("day14_data.txt")
poly_ins(p_init, pairs, inserts, 10)

Polymer has an initial length of 20: PHOSBSKBBBFSPPPCCCHN

Step 1: Polymer has a length of 39
Step 2: Polymer has a length of 77
Step 3: Polymer has a length of 153
Step 4: Polymer has a length of 305
Step 5: Polymer has a length of 609
Step 6: Polymer has a length of 1217
Step 7: Polymer has a length of 2433
Step 8: Polymer has a length of 4865
Step 9: Polymer has a length of 9729
Step 10: Polymer has a length of 19457

Most common element is N with 3661 entries.
The least common element is B with 787 entries.
The solution is: 3661 - 787 = 2874.


## Part 2

The resulting polymer isn't nearly strong enough to reinforce the submarine. You'll need to run more steps of the pair insertion process; a total of 40 steps should do it.

In the above example, the most common element is ```B``` (occurring ```2192039569602``` times) and the least common element is ```H``` (occurring ```3849876073``` times); subtracting these produces ```2188189693529```.

Apply 40 steps of pair insertion to the polymer template and find the most and least common elements in the result. What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?

In [6]:
p_init, pairs, inserts = load_insertions("day14_data_test.txt")
poly_ins(p_init, pairs, inserts, 40)

Polymer has an initial length of 4: NNCB

Step 1: Polymer has a length of 7
Step 2: Polymer has a length of 13
Step 3: Polymer has a length of 25
Step 4: Polymer has a length of 49
Step 5: Polymer has a length of 97
Step 6: Polymer has a length of 193
Step 7: Polymer has a length of 385
Step 8: Polymer has a length of 769
Step 9: Polymer has a length of 1537
Step 10: Polymer has a length of 3073
Step 11: Polymer has a length of 6145
Step 12: Polymer has a length of 12289
Step 13: Polymer has a length of 24577
Step 14: Polymer has a length of 49153
Step 15: Polymer has a length of 98305
Step 16: Polymer has a length of 196609
Step 17: Polymer has a length of 393217
Step 18: Polymer has a length of 786433
Step 19: Polymer has a length of 1572865
Step 20: Polymer has a length of 3145729
Step 21: Polymer has a length of 6291457
Step 22: Polymer has a length of 12582913
Step 23: Polymer has a length of 25165825
Step 24: Polymer has a length of 50331649
Step 25: Polymer has a length of 10066

In [7]:
p_init, pairs, inserts = load_insertions("day14_data.txt")
poly_ins(p_init, pairs, inserts, 40)

Polymer has an initial length of 20: PHOSBSKBBBFSPPPCCCHN

Step 1: Polymer has a length of 39
Step 2: Polymer has a length of 77
Step 3: Polymer has a length of 153
Step 4: Polymer has a length of 305
Step 5: Polymer has a length of 609
Step 6: Polymer has a length of 1217
Step 7: Polymer has a length of 2433
Step 8: Polymer has a length of 4865
Step 9: Polymer has a length of 9729
Step 10: Polymer has a length of 19457
Step 11: Polymer has a length of 38913
Step 12: Polymer has a length of 77825
Step 13: Polymer has a length of 155649
Step 14: Polymer has a length of 311297
Step 15: Polymer has a length of 622593
Step 16: Polymer has a length of 1245185
Step 17: Polymer has a length of 2490369
Step 18: Polymer has a length of 4980737
Step 19: Polymer has a length of 9961473
Step 20: Polymer has a length of 19922945
Step 21: Polymer has a length of 39845889
Step 22: Polymer has a length of 79691777
Step 23: Polymer has a length of 159383553
Step 24: Polymer has a length of 318767105
St